# Libraries

In [1]:
import pandas as pd
import requests
from shapely import wkt
from geopy.distance import geodesic
import numpy as np
from scipy.spatial.distance import squareform, pdist
import random
from itertools import permutations
import json
import folium

pd.options.display.max_columns = None
pd.options.display.max_rows = 20

# Helpful functions

In [2]:
# API key to gain access to Open Cage Geo Data
API_KEY = '1232cd7984e543e188eebab0f8d6956f'

# Function to get coordinates
def get_coordinates(county, name, state="Florida"):
    place_name = f"{name}, {county} County, {state}"
    url = f"https://api.opencagedata.com/geocode/v1/json?q={place_name}&key={API_KEY}"
    response = requests.get(url)
    data = response.json()
    if data['results']:
        location = data['results'][0]['geometry']
        return (location['lat'], location['lng'])
    else:
        return (None, None)

In [17]:
def preprocess(file_path):
    """
    Preprocesses csv dataset.
    """
    # Load dataset
    ds = pd.read_csv(file_path)
    
    # Make copy of dataframe
    df = ds.copy()
    
    # Preprocessing steps - dropping null values
    df.drop(columns=['created_user', 'created_date', 'last_edited_user', 'last_edited_date', 'WKT'], inplace = True)
    df.dropna(inplace=True)
    
    # Add latitude and longitude data
    df['latitude'] = None
    df['longitude'] = None
    for index, row in df.iterrows():
        county = row['COUNTY']
        name = row['NAME']
        coordinates = get_coordinates(county, name)
        df.at[index, 'latitude'] = coordinates[0]
        df.at[index, 'longitude'] = coordinates[1]
    
    # Zip this data into coordinates
    df['coordinates'] = list(zip(df['latitude'], df['longitude']))
    df = df.drop(['latitude', 'longitude'], axis=1)
    
    # Deal with duplicates
    filtered_df = df.drop([63, 30, 113, 109, 85, 147, 32]) # Duplicate names, not actually beach locations
    filtered_df.drop_duplicates(inplace=True)
    filtered_df = filtered_df[filtered_df['NAME']!='UNSURVEYED'] # Missing name
    filtered_df = filtered_df[filtered_df['COUNTY']!= 'SARASOAT'] # Typo
    filtered_df = filtered_df.drop_duplicates(subset='coordinates') # Drop duplicate coordinates
    
    # Save preprocessed dataset to backup csv
    filtered_df.reset_index(drop=True)
    filtered_df.to_csv('updated2_beaches.csv', index=False)

    return filtered_df

In [4]:
# Function to calculate the nearest n beaches from the starting beach
def calculate_nearest_beaches(df, starting_beach, n):
    """
    Calculate nearest n beaches from starting beach, using geodesic distance between lat/long coordinates.
    
    Inputs: 
        - df: DataFrame of all beach data, 
        - starting_beach: string, precise name of starting beach according to provided df
        - n: int, number of nearest beaches to calculate

    Output:
        - DataFrame containing name of beach and geodesic distance (in miles) from starting beach
    """
    starting_beach_coord = df[df.NAME.str.upper() == starting_beach.upper()].coordinates.iloc[0]
    df = df.copy()
    df['geodesic_distance'] = df['coordinates'].apply(lambda x: geodesic(starting_beach_coord, x).miles)
    df = df.sort_values(by='geodesic_distance', ascending=True).head(n + 1)
    return df[['NAME', 'coordinates', 'geodesic_distance']]

In [5]:
# Function to calculate the distance matrix
def calculate_distance_matrix(df):
    """
    Calculate distance matrix between every point in a dataframe.

    Input:
        - Dataframe, including coordinates
    Output:
        - Distance matrix (list of lists, including distance from each point to every other point)
    """
    coords = df['coordinates'].tolist()
    distance_matrix = squareform(pdist(coords, lambda u, v: geodesic(u, v).miles))
    return distance_matrix

In [6]:
# Nearest Neighbor Algorithm to find the optimal route and calculate total distance
def nearest_neighbor_algorithm(distance_matrix):
    """
    Uses Nearest Neighbor algorithm to find a good route.

    Input: 
        - Distance matrix (list of lists, including distance from each point to every other point)
    Output:
        - Route (list of location names)
        - Total distance (miles)
        - Distances (list of distances between each location, miles)
    """
    n = len(distance_matrix)
    visited = [False] * n
    route = [0]  # Start from the initial location
    visited[0] = True
    total_distance = 0.0
    distances = []

    for _ in range(1, n):
        last_visited = route[-1]
        next_city = np.argmin([distance_matrix[last_visited][j] if not visited[j] else float('inf') for j in range(n)])
        route.append(next_city)
        visited[next_city] = True
        distance = round(distance_matrix[last_visited][next_city], 2)
        distances.append(distance)
        total_distance += distance

    return route, total_distance, distances

In [7]:
def calculate_random_route(distance_matrix):
    """
    Uses randomization to calculate a baseline route.

    Input: 
        - Distance matrix (list of lists, including distance from each point to every other point)
    Output:
        - Route (list of location names)
        - Total distance (miles)
        - Distances (list of distances between each location, miles)    
    """
    n = len(distance_matrix)
    route = list(range(1, n))  # Start from the second location
    random.shuffle(route)
    route = [0] + route  # Add the starting location at the beginning
    total_distance = 0.0
    distances = []

    for i in range(n - 1):
        distance = distance_matrix[route[i]][route[i + 1]]
        distances.append(f"{distance:.2f}")
        total_distance += distance

    return route, total_distance, distances

In [8]:
# Brute Force Algorithm to find the optimal route and calculate total distance
def calculate_brute_force_route(distance_matrix):
    """
    Uses a brute force approach to find the best route.

    Input: 
        - Distance matrix (list of lists, including distance from each point to every other point)
    Output:
        - Route (list of location names)
        - Total distance (miles)
        - Distances (list of distances between each location, miles)   
    """
    n = len(distance_matrix)
    min_distance = float('inf')
    best_route = None
    best_distances = []

    for perm in permutations(range(1, n)):
        current_route = [0] + list(perm)
        current_distance = 0.0
        distances = []

        for i in range(n - 1):
            distance = distance_matrix[current_route[i]][current_route[i + 1]]
            distances.append(f"{distance:.2f}")
            current_distance += distance

        if current_distance < min_distance:
            min_distance = current_distance
            best_route = current_route
            best_distances = distances

    return best_route, min_distance, best_distances

In [11]:
# Main function to call the other functions and get the optimal route and total distance
def calculate_route(df, starting_beach, n, algorithm='nearest neighbors'):
    """
    Calculate optimal route from start to finish, calling other functions. Choose between performance/efficiency levels.

    Inputs: 
        - df: DataFrame of all location data
        - starting_beach: string, precise name of starting location/beach according to provided df
        - n: int, number of nearest locations to calculate. Refer to algorithm parameter for guidance on maximum size of n.
        - algorithm: string, one of three options:
            - 'nearest neighbors': best value of performance and efficiency (n has no upper limit)
            - 'random': most efficient, but poor performance (baseline model, n has no upper limit)
            - 'brute force': highest performance, lowest efficiency (n cannot exceed 9 without massive performance loss)
    Outputs:
        - optimal route: ordered list of locations, beginning with starting location
        - total distance: float, total distance traveled from starting location to final location, miles
        - distances: ordered list of distances between each location, miles
    """
    nearest_beaches = calculate_nearest_beaches(df, starting_beach, n)
    distance_matrix = calculate_distance_matrix(nearest_beaches)
    
    if algorithm.lower() == 'nearest neighbors':
        route_indices, total_distance, distances = nearest_neighbor_algorithm(distance_matrix)
    elif algorithm.lower() == 'random':
        route_indices, total_distance, distances = calculate_random_route(distance_matrix)
    elif algorithm.lower() == 'brute force':
        route_indices, total_distance, distances = calculate_brute_force_route(distance_matrix)
    
    optimal_route = nearest_beaches.iloc[route_indices]
    
    route = optimal_route.NAME.tolist()
    coordinates = optimal_route.coordinates.tolist()
    distances = [float(d) for d in distances]  # Convert distances to float
    
    result = {
        'optimal_route_sequence': route,
        'distances_between_beaches': distances,
        'total_distance': round(total_distance, 2),
        'coordinates': coordinates
    }
    
    return json.dumps(result, indent=4), optimal_route

In [12]:
def plot_route(optimal_route):
    """
    Plots the optimal route on an interactive map.
    
    Inputs:
        - optimal_route: DataFrame containing the optimal route with names and coordinates
    """
    # Initialize the map centered around Florida
    m = folium.Map(location=[27.9944024, -81.7602544], zoom_start=6)
    
    # Add markers and lines for the optimal route
    points = []
    for idx, row in optimal_route.iterrows():
        folium.Marker(
            location=[row['coordinates'][0], row['coordinates'][1]],
            popup=f"{row['NAME']}<br>Coordinates: {row['coordinates']}"
        ).add_to(m)
        points.append((row['coordinates'][0], row['coordinates'][1]))
    
    folium.PolyLine(points, color="blue", weight=2.5, opacity=1).add_to(m)
    
    return m

# Main Code

In [18]:
# Load the filtered DataFrame
filtered_df = preprocess('florida-beach-names.csv')

In [19]:
# User input for starting beach and number of additional beaches
starting_beach = input("Enter the starting beach: ")
n = int(input("Enter the number of additional beaches: "))

# Calculate the route using the desired algorithm
result_json, optimal_route = calculate_route(filtered_df, starting_beach, n, algorithm='nearest neighbors')

# Print the JSON result
print(result_json)
print(optimal_route)

# Plot and save the route map
route_map = plot_route(optimal_route)
route_map.save('route_map.html')

# Display the route map (if using a Jupyter Notebook)
display(route_map)

Enter the starting beach:  hanna park
Enter the number of additional beaches:  9


{
    "optimal_route_sequence": [
        "HANNA PARK",
        "PONTE VEDRA N",
        "PONTE VEDRA S",
        "S COUNTY BCHS",
        "L TALBOT ISL SP",
        "ST VINCENT NWR",
        "DOG ISL",
        "ST GEO ISL SP",
        "ST GEO ISL",
        "FT CLINCH SP"
    ],
    "distances_between_beaches": [
        9.1,
        15.42,
        36.43,
        0.41,
        9.43,
        8.89,
        0.05,
        1.6,
        21.0
    ],
    "total_distance": 102.33,
    "coordinates": [
        [
            30.3709553,
            -81.4028428
        ],
        [
            30.2396865,
            -81.3856384
        ],
        [
            30.0224671,
            -81.3236866
        ],
        [
            30.4686,
            -81.650746
        ],
        [
            30.474452,
            -81.65222
        ],
        [
            30.573019,
            -81.542569
        ],
        [
            30.678079,
            -81.456019
        ],
        [
            30.67799